In [1]:
import faiss
exec(open("../database.py").read())
exec(open("../helperFunctions.py").read())
import numpy as np
from keras.datasets import mnist
from time import perf_counter
from ast import literal_eval

In [2]:
name = 'glove-100-angular'
nameFull = name +'-true-labels.xlsx'
datasetTrainImages, datasetTestImages, _ = get_ann_benchmark_data(name)

<string>:57: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.


trainDataset :  (1183514, 100)
testDataset :  (10000, 100)


<string>:58: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
<string>:59: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.


***Create faiss***

In [3]:
def createIndex(indexMethod, datasetImages):
    d = datasetImages.shape[1] # dimension
    M = 16
    time_start = perf_counter()
    index = indexMethod(d, M)
    index.add(datasetImages) 
    time_end = perf_counter()
    totalTime = (time_end - time_start)
    print(f'Took {totalTime:.3f} seconds')
    return (index, totalTime)
(indexedStruct, time) = createIndex(faiss.IndexHNSWFlat, datasetTrainImages)

Took 90.438 seconds


In [4]:
# (min, max) = createIndexNumerous(createIndex, AnnoyIndex, datasetImages, 10)
# print('min : ', min, '\n','max : ', max,)

In [5]:
indexes = []
distances = []
def measureTime(par, indexes, distances, datasetImages):
    k=100
    totalTime = 0
    for i in range(par) : 
        xq = datasetImages[i:i+1].astype('float32') # Use the first image as the query vector
        time_start = perf_counter()
        distance, index = indexedStruct.search(xq, k) 
        time_end = perf_counter()
        totalTime += (time_end - time_start)
        distances.append(np.sqrt(distance[0]))
        indexes.append(index[0])
    # report the duration
    print(f'Took {totalTime:.3f} seconds')
measureTime(1000, indexes, distances, datasetTestImages)

Took 2.090 seconds


In [6]:
# (min, max) = measureTimeNumerous(measureTime, 10)
# print('min : ', min, '\n','max : ', max,)

In [7]:
indexes = np.array(indexes)
distances = np.round(np.array(distances).astype(float), 4)

In [8]:
print('indexes : ', indexes.shape)
print('distances : ', distances.shape)

indexes :  (1000, 100)
distances :  (1000, 100)


In [9]:
path = '../datasets/'+nameFull
(trueIndexes, trueDistances) = readDB(path)

trueIndexes :  (1000, 100)
trueDistances :  (1000, 100)


In [10]:
compareFirstTen(indexes, distances, trueIndexes, trueDistances)

660281 || 660281
3.8792 || 3.8792
1133489 || 93438
3.9021 || 3.8838
989330 || 1133489
3.9523 || 3.9021
242797 || 97478
3.9568 || 3.9037
256447 || 995275
3.9857 || 3.9294
914908 || 207076
3.9933 || 3.9378
956338 || 989330
4.0067 || 3.9523
859959 || 846101
4.009 || 3.955
861914 || 242797
4.0619 || 3.9568
134899 || 402216
4.073 || 3.9697


In [11]:
calculateRecallAverage(indexes, distances, trueIndexes, trueDistances)
calculateRecallAverage(indexes, distances, trueIndexes, trueDistances, 1.01)
calculateRecallAverage(indexes, distances, trueIndexes, trueDistances, 1.1)

0.0190
0.0593
0.9199


In [12]:
print(indexes[0]) # will find the 100 nearest neighbors

[ 660281 1133489  989330  242797  256447  914908  956338  859959  861914
  134899  294110  264602  499677  834699  660541  368655  583340  979282
 1034304  599119  639351  896732  783290   21236 1169856  491578  699013
  290452 1182920 1025415 1071246  615918  797936  906022  547334  640074
  332959  459704  723274  423010  586689  486884  519061  712479 1110193
 1043584  221021  623971  645527 1043180 1147765  734899  519422  138345
  625955  161775  272975  487872  368290  899723  970642   43043 1001416
  114867 1069601  594135  553449   75808  477174  215730  959078  910564
  381903   33350  753828  554158  691462  538779  876781  198052  238774
  907567  265909 1099859  604428    8688  740389  592918   33577  723857
  468774  283506 1146265  278015  802493  189966  713982  491668   21276
  622365]


In [13]:
# draw_mnist(indexes, distances, datasetTrainImages)